In [1]:
##Actualizando...
# Parámetros de la API
# Obtener la fecha actual
import requests
from datetime import datetime, timedelta
import pandas as pd
import time

In [24]:
##Actualizando...
# Parámetros de la API
# Obtener la fecha actual
import requests
from datetime import datetime, timedelta
import pandas as pd
import time


# Parámetros de la API
base_url = "https://trading.mav-sa.com.ar/cgi-bin/wspd_cgi.sh/WService=wsbroker1/cpd-concertacion-csv_v8.r"
user_id = "avsawebservice"
password = "avsa1601"

# Obtener la fecha actual
fecha_actual = datetime.now()

# Formato de fecha para la API (DD/MM/AA)
fecha_formato_api = "%d/%m/%y"

# Crear una lista para almacenar los datos
datos = []

# Bucle para generar fechas desde el principio del año actual hasta la fecha actual
fecha_actual = fecha_actual.replace(hour=0, minute=0, second=0, microsecond=0)  # Ajustar a la medianoche
fecha = fecha_actual
while fecha <= fecha_actual:
    # Construir la fecha en formato de texto
    fecha_texto = fecha.strftime(fecha_formato_api)

    # Construir la URL de la solicitud con la fecha actual
    url = f"{base_url}?mode=ws&an={user_id}&id=avsawebservice&password={password}&fecha=23/04/24"

    # Realizar la solicitud HTTP
    requests.packages.urllib3.disable_warnings()  # Desactivar la advertencia sobre solicitud HTTPS no verificada
    response = requests.get(url, verify=False)

    # Procesar la respuesta y almacenar los datos en la lista
    datos.append([fecha_texto] + response.text.strip().split(','))

    # Avanzar al siguiente día
    fecha += timedelta(days=1)

    # Esperar 60 segundos antes de la próxima ejecución
    time.sleep(60)

# Ahora puedes trabajar directamente con la lista de datos para realizar transformaciones
# Por ejemplo, puedes convertir la lista de datos en un DataFrame de pandas y aplicar transformaciones
df = pd.DataFrame(datos)
# Realizar transformaciones adicionales en el DataFrame si es necesario
print(df)

         0                                                  1   \
0  24/04/24  Subasta; Segmento; Moneda; C-V; Tasa; Contrapa...   

                                                  2          3       4     5   \
0  00;427;14:07:57;23/04/2024;24/04/2024;1611988;...  00;117931  37;179  35;0   

                                                  6   \
0  00;240117;30-71213850-1;13:51:37;;RI;*MAV15060...   

                                                  7   \
0  66;30-70850558-3;;427;AO;;;;MADA S.A.\r\n10602...   

                                                  8          9   ...      28  \
0  00;427;14:07:57;23/04/2024;24/04/2024;1611988;...  00;117931  ...  04;339   

     29                                                 30  \
0  26;0  00;310600;30-67880623-0;16:35:49;;EX;*FED13050...   

                                        31  \
0  24;;;501;;;;;\r\n1060375;Avalado;$;V;52   

                                                  32         33      34    35  \
0  00;567;15:17:37

In [25]:
import csv

fecha_hoy = datetime.now().strftime("%Y-%m-%d")

# Generar el nombre del archivo CSV con la fecha de hoy
nombre_archivo = f"datos_{fecha_hoy}.csv"

# Escribir los datos en el archivo CSV
with open(nombre_archivo, "w", newline="", encoding="utf-8") as archivo:
    escritor_csv = csv.writer(archivo)
    escritor_csv.writerows(datos)

# Leer los datos del archivo CSV y crear un DataFrame
df = pd.read_csv(nombre_archivo, delimiter=';', quotechar='"', lineterminator='\n')

# Mostrar el DataFrame
print(df)

   24/04/24,"Subasta        Segmento  Moneda  C-V    Tasa   Contraparte  \
0            1060280  No Garantizado       $    C  80",00           427   
1            1060280  No Garantizado       $    V  80",00           427   
2            1060284         Avalado       $    V  50",00           449   
3            1060365         Avalado       $    V  55",00           501   
4            1060365         Avalado       $    V  55",00           501   
5            1060375         Avalado       $    V  52",00           567   

   Hora Conc.  Concertación  Liquidación   Reg.Oper.  ...  Nro.Cta.Libr.  \
0    14:07:57    23/04/2024   24/04/2024     1611988  ...        15846.0   
1    14:07:57    23/04/2024   24/04/2024     1611988  ...        15846.0   
2    14:35:09    23/04/2024   24/04/2024     1612065  ...            NaN   
3    16:37:50    23/04/2024   24/04/2024     1612489  ...            NaN   
4    16:37:50    23/04/2024   24/04/2024     1612490  ...            NaN   
5    15:17:37    2

In [86]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 47 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   23/04/24,"Subasta       6 non-null      int64  
 1    Segmento               6 non-null      object 
 2    Moneda                 6 non-null      object 
 3    C-V                    6 non-null      object 
 4    Tasa                   6 non-null      object 
 5    Contraparte            6 non-null      int64  
 6    Hora Conc.             6 non-null      object 
 7    Concertación           6 non-null      object 
 8    Liquidación            6 non-null      object 
 9    Reg.Oper.              6 non-null      int64  
 10   Id.Cheque              6 non-null      int64  
 11   Banco                  6 non-null      int64  
 12   Nro.Cheque             6 non-null      int64  
 13   Acreditación           6 non-null      object 
 14   Monto                  6 non-null      object

In [26]:
df = df.astype(str)


In [27]:
# Función para extraer la fecha y otros valores de la primera columna
def separar_fecha_valor(row):
    valores = row[0].split(',')
    fecha = valores[0].strip()
    otros_valores = ','.join(valores[2:]).strip()
    return pd.Series([fecha, otros_valores])

# Aplicar la función a cada fila del DataFrame y asignar los resultados a nuevas columnas
df[['Fecha', 'Subasta']] = df.apply(separar_fecha_valor, axis=1)

# Mostrar el DataFrame resultante
print(df)


  24/04/24,"Subasta        Segmento  Moneda  C-V    Tasa  Contraparte  \
0           1060280  No Garantizado       $    C  80",00          427   
1           1060280  No Garantizado       $    V  80",00          427   
2           1060284         Avalado       $    V  50",00          449   
3           1060365         Avalado       $    V  55",00          501   
4           1060365         Avalado       $    V  55",00          501   
5           1060375         Avalado       $    V  52",00          567   

   Hora Conc.  Concertación  Liquidación  Reg.Oper.  ... CUIT Librador/Deudor  \
0    14:07:57    23/04/2024   24/04/2024    1611988  ...        30-70850558-3   
1    14:07:57    23/04/2024   24/04/2024    1611988  ...        30-70850558-3   
2    14:35:09    23/04/2024   24/04/2024    1612065  ...                  nan   
3    16:37:50    23/04/2024   24/04/2024    1612489  ...                  nan   
4    16:37:50    23/04/2024   24/04/2024    1612490  ...                  nan   
5 

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_16620\2368009279.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  valores = row[0].split(',')


In [28]:
# Supongamos que tienes un DataFrame llamado df con una columna 'fecha'

# Define una función para procesar los valores de la columna 'fecha'
def procesar_fecha(valor):
    if "/" in str(valor):
        return valor
    else:
        return None  # O cualquier otro valor que desees poner en la columna 'otros_valores'

# Aplica la función a la columna 'fecha' y guarda los resultados en una nueva columna 'fecha_procesada'
df['Fecha'] = df['Fecha'].apply(procesar_fecha)

# Crea una nueva columna 'otros_valores' con los valores que no contenían '/'
df['Subasta'] = df['Fecha'].apply(lambda x: x if "/" not in str(x) else None)

In [29]:
df['Fecha'] = df['Fecha'].fillna(df[' Concertación'])


In [30]:
df

,"24/04/24,""Subasta",Segmento,Moneda,C-V,Tasa,Contraparte,Hora Conc.,Concertación,Liquidación,Reg.Oper.,...,CUIT Librador/Deudor,CUIT PyME,Liquidador Compra,Caracter,CUIT Benef.,Razón Benef.,Cond. Pyme,Razón Librador/Deudor\r,Fecha,Subasta
0,1060280,No Garantizado,$,C,"80"",00",427,14:07:57,23/04/2024,24/04/2024,1611988,...,30-70850558-3,nan,427,AO,nan,nan,nan,MADA S.A.\r,23/04/2024,None
1,1060280,No Garantizado,$,V,"80"",00",427,14:07:57,23/04/2024,24/04/2024,1611988,...,30-70850558-3,nan,427,AO,nan,nan,nan,MADA S.A.\r,23/04/2024,None
2,1060284,Avalado,$,V,"50"",00",449,14:35:09,23/04/2024,24/04/2024,1612065,...,nan,nan,449,nan,nan,nan,nan,\r,23/04/2024,None
3,1060365,Avalado,$,V,"55"",00",501,16:37:50,23/04/2024,24/04/2024,1612489,...,nan,nan,501,nan,nan,nan,nan,\r,23/04/2024,None
4,1060365,Avalado,$,V,"55"",00",501,16:37:50,23/04/2024,24/04/2024,1612490,...,nan,nan,501,nan,nan,nan,nan,\r,23/04/2024,None
5,1060375,Avalado,$,V,"52"",00",567,15:17:37,23/04/2024,24/04/2024,1612235,...,nan,nan,567,nan,nan,nan,nan,\r,23/04/2024,None


In [31]:
primera_columna_duplicada = df.iloc[:, 0]

# Crear la columna 'otros_valores'
df['Subasta'] = primera_columna_duplicada.apply(lambda x: x if '/' not in str(x) else None)

In [32]:
df

,"24/04/24,""Subasta",Segmento,Moneda,C-V,Tasa,Contraparte,Hora Conc.,Concertación,Liquidación,Reg.Oper.,...,CUIT Librador/Deudor,CUIT PyME,Liquidador Compra,Caracter,CUIT Benef.,Razón Benef.,Cond. Pyme,Razón Librador/Deudor\r,Fecha,Subasta
0,1060280,No Garantizado,$,C,"80"",00",427,14:07:57,23/04/2024,24/04/2024,1611988,...,30-70850558-3,nan,427,AO,nan,nan,nan,MADA S.A.\r,23/04/2024,1060280
1,1060280,No Garantizado,$,V,"80"",00",427,14:07:57,23/04/2024,24/04/2024,1611988,...,30-70850558-3,nan,427,AO,nan,nan,nan,MADA S.A.\r,23/04/2024,1060280
2,1060284,Avalado,$,V,"50"",00",449,14:35:09,23/04/2024,24/04/2024,1612065,...,nan,nan,449,nan,nan,nan,nan,\r,23/04/2024,1060284
3,1060365,Avalado,$,V,"55"",00",501,16:37:50,23/04/2024,24/04/2024,1612489,...,nan,nan,501,nan,nan,nan,nan,\r,23/04/2024,1060365
4,1060365,Avalado,$,V,"55"",00",501,16:37:50,23/04/2024,24/04/2024,1612490,...,nan,nan,501,nan,nan,nan,nan,\r,23/04/2024,1060365
5,1060375,Avalado,$,V,"52"",00",567,15:17:37,23/04/2024,24/04/2024,1612235,...,nan,nan,567,nan,nan,nan,nan,\r,23/04/2024,1060375


In [33]:


# Eliminar las dos primeras columnas
df = df.drop(df.columns[[0]], axis=1)
df = df.drop_duplicates()

# Obtener una lista con el nombre de todas las columnas
columnas = df.columns.tolist()

# Reorganizar la lista de columnas para que "Fecha" y "Otros_Valores" estén al principio
columnas = ['Fecha', 'Subasta'] + [col for col in columnas if col not in ['Fecha', 'Subasta']]

# Reorganizar el DataFrame con las nuevas columnas
df = df[columnas]




In [34]:
df

,Fecha,Subasta,Segmento,Moneda,C-V,Tasa,Contraparte,Hora Conc.,Concertación,Liquidación,...,Nro.Cta.Libr.,IVA Der.Mer.,CUIT Librador/Deudor,CUIT PyME,Liquidador Compra,Caracter,CUIT Benef.,Razón Benef.,Cond. Pyme,Razón Librador/Deudor\r
0,23/04/2024,1060280,No Garantizado,$,C,"80"",00",427,14:07:57,23/04/2024,24/04/2024,...,15846.0,"37,""66",30-70850558-3,nan,427,AO,nan,nan,nan,MADA S.A.\r
1,23/04/2024,1060280,No Garantizado,$,V,"80"",00",427,14:07:57,23/04/2024,24/04/2024,...,15846.0,"75,""33",30-70850558-3,nan,427,AO,nan,nan,nan,MADA S.A.\r
2,23/04/2024,1060284,Avalado,$,V,"50"",00",449,14:35:09,23/04/2024,24/04/2024,...,nan,"39,""44",nan,nan,449,nan,nan,nan,nan,\r
3,23/04/2024,1060365,Avalado,$,V,"55"",00",501,16:37:50,23/04/2024,24/04/2024,...,nan,"51,""34",nan,nan,501,nan,nan,nan,nan,\r
4,23/04/2024,1060365,Avalado,$,V,"55"",00",501,16:37:50,23/04/2024,24/04/2024,...,nan,"71,""24",nan,nan,501,nan,nan,nan,nan,\r
5,23/04/2024,1060375,Avalado,$,V,"52"",00",567,15:17:37,23/04/2024,24/04/2024,...,nan,"151,04",nan,nan,567,nan,nan,nan,nan,\r


In [35]:
from datetime import datetime

# Convertir 'Fecha' a tipo datetime con el formato adecuado
df['Fecha'] = pd.to_datetime(df['Fecha'], format='%d/%m/%Y')

# Convertir 'Vto.' a tipo datetime con el formato adecuado
df['Vto.'] = pd.to_datetime(df['Vto.'], format='%d/%m/%Y')
df['dias_entre_fechas'] = (df['Vto.'] - df['Fecha']).dt.days

In [36]:
# Eliminar las comas de las cadenas y convertirlas a números de punto flotante
df[' Monto'] = df[' Monto'].str.split(',').str[0]
# Define una función para asignar la categoría en función de las palabras clave
def asignar_categoria(texto):
    if 'ARP' in texto:
        return 'ARPY'
    elif 'GAR' in texto:
        return 'GAR'
    elif 'MAV' in texto:
        return 'MAV'
    elif 'ACE' in texto:
        return 'ACEN'
    elif 'FDE' in texto:
        return 'FIDEM'
    elif 'NOV' in texto:
        return 'NOV'
    elif 'CRE' in texto:
        return 'CREC'
    elif 'POT' in texto:
        return 'POT'
    elif 'ACI' in texto:
        return 'ACIN'
    elif 'POT' in texto:
        return 'POT'
    elif 'BIN' in texto:
        return 'BIND'
    elif '#UGA' in texto:
        return 'GAR'
    elif '#UFA' in texto:
        return 'AVFE'
    elif '#UAR' in texto:
        return 'ARP'
    elif '#UAM' in texto:
        return 'AMER' 
    elif '#UAC' in texto:
        return 'ACIN'
    elif 'FED' in texto:
        return 'FEDE'
    
    
    # AgregaMOS más condiciones según sea necesario
    else:
        return 'otra_categoria'

# Aplica la función a la columna 'texto' para crear la nueva columna 'SGR'
df['SGR'] = df[' Cod.Ch.'].apply(asignar_categoria)

# Muestra el DataFrame con la nueva columna
print(df)

       Fecha  Subasta        Segmento  Moneda  C-V    Tasa  Contraparte  \
0 2024-04-23  1060280  No Garantizado       $    C  80",00          427   
1 2024-04-23  1060280  No Garantizado       $    V  80",00          427   
2 2024-04-23  1060284         Avalado       $    V  50",00          449   
3 2024-04-23  1060365         Avalado       $    V  55",00          501   
4 2024-04-23  1060365         Avalado       $    V  55",00          501   
5 2024-04-23  1060375         Avalado       $    V  52",00          567   

   Hora Conc.  Concertación  Liquidación  ... CUIT Librador/Deudor CUIT PyME  \
0    14:07:57    23/04/2024   24/04/2024  ...        30-70850558-3       nan   
1    14:07:57    23/04/2024   24/04/2024  ...        30-70850558-3       nan   
2    14:35:09    23/04/2024   24/04/2024  ...                  nan       nan   
3    16:37:50    23/04/2024   24/04/2024  ...                  nan       nan   
4    16:37:50    23/04/2024   24/04/2024  ...                  nan       n

In [114]:
# Configurar la opción para mostrar todas las columnas
pd.set_option('display.max_columns', None)

# Ahora puedes imprimir el DataFrame y ver todas las columnas
print(df)

       Fecha  Subasta        Segmento  Moneda  C-V    Tasa  Contraparte  \
0 2024-04-23  1060280  No Garantizado       $    C  80",00          427   
1 2024-04-23  1060280  No Garantizado       $    V  80",00          427   
2 2024-04-23  1060284         Avalado       $    V  50",00          449   
3 2024-04-23  1060365         Avalado       $    V  55",00          501   
4 2024-04-23  1060365         Avalado       $    V  55",00          501   
5 2024-04-23  1060375         Avalado       $    V  52",00          567   

   Hora Conc.  Concertación  Liquidación  Reg.Oper.  Id.Cheque  Banco  \
0    14:07:57    23/04/2024   24/04/2024    1611988    1602622      7   
1    14:07:57    23/04/2024   24/04/2024    1611988    1602622      7   
2    14:35:09    23/04/2024   24/04/2024    1612065    1601387    285   
3    16:37:50    23/04/2024   24/04/2024    1612489    1603084     11   
4    16:37:50    23/04/2024   24/04/2024    1612490    1603085     11   
5    15:17:37    23/04/2024   24/04/

In [37]:
df

,Fecha,Subasta,Segmento,Moneda,C-V,Tasa,Contraparte,Hora Conc.,Concertación,Liquidación,...,CUIT Librador/Deudor,CUIT PyME,Liquidador Compra,Caracter,CUIT Benef.,Razón Benef.,Cond. Pyme,Razón Librador/Deudor\r,dias_entre_fechas,SGR
0,2024-04-23,1060280,No Garantizado,$,C,"80"",00",427,14:07:57,23/04/2024,24/04/2024,...,30-70850558-3,nan,427,AO,nan,nan,nan,MADA S.A.\r,53,MAV
1,2024-04-23,1060280,No Garantizado,$,V,"80"",00",427,14:07:57,23/04/2024,24/04/2024,...,30-70850558-3,nan,427,AO,nan,nan,nan,MADA S.A.\r,53,MAV
2,2024-04-23,1060284,Avalado,$,V,"50"",00",449,14:35:09,23/04/2024,24/04/2024,...,nan,nan,449,nan,nan,nan,nan,\r,36,GAR
3,2024-04-23,1060365,Avalado,$,V,"55"",00",501,16:37:50,23/04/2024,24/04/2024,...,nan,nan,501,nan,nan,nan,nan,\r,14,FEDE
4,2024-04-23,1060365,Avalado,$,V,"55"",00",501,16:37:50,23/04/2024,24/04/2024,...,nan,nan,501,nan,nan,nan,nan,\r,20,FEDE
5,2024-04-23,1060375,Avalado,$,V,"52"",00",567,15:17:37,23/04/2024,24/04/2024,...,nan,nan,567,nan,nan,nan,nan,\r,29,ARPY


In [38]:
df[' Tasa'] = df[' Tasa'].str.replace('"', '')

# Reemplazar la coma por un punto en los valores de la columna "tasa"
df[' Tasa'] = df[' Tasa'].str.replace(',', '.')

# Convertir la columna "tasa" a tipo float
df[' Tasa'] = df[' Tasa'].astype(float)

# Redondear los valores en la columna "tasa" al entero más cercano
df[' Tasa'] = df[' Tasa'].round()

In [39]:
columnas_a_ignorar = ['Fecha', 'Vto.',' Hora Conc.',' Cod.Ch.','SGR',' Moneda Liquidacion','Nombre Banco','Razón Librador/Deudor\r','Cond. Pyme', ' Banco',' C-V',' Hora Ord.',' Hora Conc.', ' Concertación', ' Liquidación', ' Reg.Oper.',' Cond.',' Banco',' Acreditación']



for columna in df.columns:
    if columna not in columnas_a_ignorar:
        # Verifica si los valores de la columna son numéricos
        if pd.api.types.is_numeric_dtype(df[columna]):
            # Convierte la columna a tipo de datos Int64
            df[columna] = df[columna].astype('Int64')

# Muestra el DataFrame con las columnas convertidas
print(df)


df[' Monto'] = df[' Monto'].astype(int)


       Fecha  Subasta        Segmento  Moneda  C-V   Tasa  Contraparte  \
0 2024-04-23  1060280  No Garantizado       $    C     80          427   
1 2024-04-23  1060280  No Garantizado       $    V     80          427   
2 2024-04-23  1060284         Avalado       $    V     50          449   
3 2024-04-23  1060365         Avalado       $    V     55          501   
4 2024-04-23  1060365         Avalado       $    V     55          501   
5 2024-04-23  1060375         Avalado       $    V     52          567   

   Hora Conc.  Concertación  Liquidación  ... CUIT Librador/Deudor CUIT PyME  \
0    14:07:57    23/04/2024   24/04/2024  ...        30-70850558-3       nan   
1    14:07:57    23/04/2024   24/04/2024  ...        30-70850558-3       nan   
2    14:35:09    23/04/2024   24/04/2024  ...                  nan       nan   
3    16:37:50    23/04/2024   24/04/2024  ...                  nan       nan   
4    16:37:50    23/04/2024   24/04/2024  ...                  nan       nan   
5

In [40]:
df = df.drop_duplicates("Subasta")


In [41]:
df

,Fecha,Subasta,Segmento,Moneda,C-V,Tasa,Contraparte,Hora Conc.,Concertación,Liquidación,...,CUIT Librador/Deudor,CUIT PyME,Liquidador Compra,Caracter,CUIT Benef.,Razón Benef.,Cond. Pyme,Razón Librador/Deudor\r,dias_entre_fechas,SGR
0,2024-04-23,1060280,No Garantizado,$,C,80,427,14:07:57,23/04/2024,24/04/2024,...,30-70850558-3,nan,427,AO,nan,nan,nan,MADA S.A.\r,53,MAV
2,2024-04-23,1060284,Avalado,$,V,50,449,14:35:09,23/04/2024,24/04/2024,...,nan,nan,449,nan,nan,nan,nan,\r,36,GAR
3,2024-04-23,1060365,Avalado,$,V,55,501,16:37:50,23/04/2024,24/04/2024,...,nan,nan,501,nan,nan,nan,nan,\r,14,FEDE
5,2024-04-23,1060375,Avalado,$,V,52,567,15:17:37,23/04/2024,24/04/2024,...,nan,nan,567,nan,nan,nan,nan,\r,29,ARPY


In [120]:
df

,Fecha,Subasta,Segmento,Moneda,C-V,Tasa,Contraparte,Hora Conc.,Concertación,Liquidación,Reg.Oper.,Id.Cheque,Banco,Nro.Cheque,Acreditación,Monto,Descuento,Der.Mer.,Der.Bol.,Comitente,CUIT Comprador,Hora Ord.,Warrant,Cond.,Cod.Ch.,Moneda Liquidacion,Tipo de Cambio,Sin Recurso,No a la Orden,Vto.,Nombre Banco,N. Sucursal,Pyme,Primera.Neg,Tipo Instrumento,Custodio/Registro,Echeqid,Plaza Cheque,Nro.Cta.Libr.,IVA Der.Mer.,CUIT Librador/Deudor,CUIT PyME,Liquidador Compra,Caracter,CUIT Benef.,Razón Benef.,Cond. Pyme,Razón Librador/Deudor\r,dias_entre_fechas,SGR
0,2024-04-23,1060280,No Garantizado,$,C,80,427,14:07:57,23/04/2024,24/04/2024,1611988,1602622,7,924,24/06/2024,1000000,"117931,37","179,35","0,00",240117,30-71213850-1,13:51:37,nan,RI,*MAV150600710,$,1,No,No,2024-06-15,GALICIA,344,No,Si,ECHEQ,CVSA,4OGNYLGZZ5M20L6,5220.0,15846.0,"37,""66",30-70850558-3,nan,427,AO,nan,nan,nan,MADA S.A.\r,53,MAV
2,2024-04-23,1060284,Avalado,$,V,50,449,14:35:09,23/04/2024,24/04/2024,1612065,1601387,285,44858818,31/05/2024,800000,"38591,92","187,81","0,00",340326,30-57612427-5,13:55:14,nan,EX,GARA290501119,$,1,No,No,2024-05-29,MACRO,738,No,Si,CPD,CVSA,nan,nan,nan,"39,""44",nan,nan,449,nan,nan,nan,nan,\r,36,GAR
3,2024-04-23,1060365,Avalado,$,V,55,501,16:37:50,23/04/2024,24/04/2024,1612489,1603084,11,0,09/05/2024,2500000,"55257,87","244,47","0,00",310600,30-67880623-0,16:35:49,nan,EX,*FED070500029,$,1,No,No,2024-05-07,NACION,109,No,Si,ECHEQ,CVSA,4OGNYLG3EP420L6,nan,nan,"51,""34",nan,nan,501,nan,nan,nan,nan,\r,14,FEDE
5,2024-04-23,1060375,Avalado,$,V,52,567,15:17:37,23/04/2024,24/04/2024,1612235,1603140,11,0,24/05/2024,3750000,"153704,68","719,26","0,00",310600,30-57612427-5,15:00:35,nan,EX,*ARP220500139,$,1,No,No,2024-05-22,NACION,109,No,Si,ECHEQ,CVSA,GYKNXJ0OG5K9MPR,nan,nan,"151,04",nan,nan,567,nan,nan,nan,nan,\r,29,ARPY


In [42]:
def asignar_periodo(dias_entre_fechas):
    if dias_entre_fechas < 60:
        return '0 a 60'
    elif 60 <= dias_entre_fechas < 90:
        return '60 a 90'
    elif 90 <= dias_entre_fechas < 120:
        return '90 a 120'
    elif 120 <= dias_entre_fechas < 180:
        return '120 a 180'
    elif 180 <= dias_entre_fechas < 210:
        return '180 a 210'
    elif 210 <= dias_entre_fechas < 240:
        return '210 a 240'
    elif 240 <= dias_entre_fechas < 300:
        return '240 a 300'
    else:
        return 'Más de 300'

# Aplicaa mosla función a la columna 'dias_entre_fechas' para crear la nueva columna 'periodo'
df['Periodo'] = df['dias_entre_fechas'].apply(asignar_periodo)

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_16620\929678408.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Periodo'] = df['dias_entre_fechas'].apply(asignar_periodo)


In [43]:
## Reordenar las columnas
# Para obtener el listado de columnas
columnas = df.columns
print("Listado de columnas:")
print(columnas)



Listado de columnas:
Index(['Fecha', 'Subasta', ' Segmento', ' Moneda', ' C-V', ' Tasa',
       ' Contraparte', ' Hora Conc.', ' Concertación', ' Liquidación',
       ' Reg.Oper.', ' Id.Cheque', ' Banco', ' Nro.Cheque', ' Acreditación',
       ' Monto', ' Descuento', ' Der.Mer.', ' Der.Bol.', ' Comitente',
       ' CUIT Comprador', ' Hora Ord.', ' Warrant', ' Cond.', ' Cod.Ch.',
       ' Moneda Liquidacion', ' Tipo de Cambio', ' Sin Recurso',
       ' No a la Orden', 'Vto.', 'Nombre Banco', 'N. Sucursal', 'Pyme',
       'Primera.Neg', 'Tipo Instrumento', 'Custodio/Registro', 'Echeqid',
       'Plaza Cheque', 'Nro.Cta.Libr.', 'IVA Der.Mer.', 'CUIT Librador/Deudor',
       'CUIT PyME', 'Liquidador Compra', 'Caracter', 'CUIT Benef.',
       'Razón Benef.', 'Cond. Pyme', 'Razón Librador/Deudor\r',
       'dias_entre_fechas', 'SGR', 'Periodo'],
      dtype='object')


In [45]:
# Para reordenar las columnas, puedes proporcionar un nuevo orden de columnas
# Por ejemplo, si deseas reorganizar las columnas en un nuevo orden específico
nuevo_orden = ['Fecha', 'Subasta',' Moneda' ,' C-V', ' Tasa',
       ' Contraparte', ' Hora Conc.', ' Concertación', ' Liquidación',
       ' Reg.Oper.', ' Id.Cheque', ' Banco', ' Nro.Cheque', ' Acreditación',
       ' Monto', ' Descuento', ' Der.Mer.', ' Der.Bol.', ' Comitente',
       ' CUIT Comprador', ' Hora Ord.', ' Warrant', ' Cond.', ' Cod.Ch.',
       ' Moneda Liquidacion', ' Tipo de Cambio', ' Sin Recurso',
       ' No a la Orden', 'Vto.', 'Nombre Banco', 'N. Sucursal', 'Pyme',
       'Primera.Neg', 'Tipo Instrumento', 'Custodio/Registro', 'Echeqid',
       'Plaza Cheque', 'Nro.Cta.Libr.', 'IVA Der.Mer.', 'CUIT Librador/Deudor',
       'CUIT PyME', 'Liquidador Compra', 'Caracter', 'CUIT Benef.',
       'Razón Benef.', 'Cond. Pyme', 'Razón Librador/Deudor\r',
       'dias_entre_fechas', 'SGR',' Segmento','Periodo']  # Define el nuevo orden de las columnas

# Reordena el DataFrame según el nuevo orden de columnas
df = df[nuevo_orden]

In [48]:
df.rename(columns={' Segmento': 'Segmento'}, inplace=True)
df

,Fecha,Subasta,Moneda,C-V,Tasa,Contraparte,Hora Conc.,Concertación,Liquidación,Reg.Oper.,...,Liquidador Compra,Caracter,CUIT Benef.,Razón Benef.,Cond. Pyme,Razón Librador/Deudor\r,dias_entre_fechas,SGR,Segmento,Periodo
0,2024-04-23,1060280,$,C,80,427,14:07:57,23/04/2024,24/04/2024,1611988,...,427,AO,nan,nan,nan,MADA S.A.\r,53,MAV,No Garantizado,0 a 60
2,2024-04-23,1060284,$,V,50,449,14:35:09,23/04/2024,24/04/2024,1612065,...,449,nan,nan,nan,nan,\r,36,GAR,Avalado,0 a 60
3,2024-04-23,1060365,$,V,55,501,16:37:50,23/04/2024,24/04/2024,1612489,...,501,nan,nan,nan,nan,\r,14,FEDE,Avalado,0 a 60
5,2024-04-23,1060375,$,V,52,567,15:17:37,23/04/2024,24/04/2024,1612235,...,567,nan,nan,nan,nan,\r,29,ARPY,Avalado,0 a 60


In [49]:
df.columns

Index(['Fecha', 'Subasta', ' Moneda', ' C-V', ' Tasa', ' Contraparte',
       ' Hora Conc.', ' Concertación', ' Liquidación', ' Reg.Oper.',
       ' Id.Cheque', ' Banco', ' Nro.Cheque', ' Acreditación', ' Monto',
       ' Descuento', ' Der.Mer.', ' Der.Bol.', ' Comitente', ' CUIT Comprador',
       ' Hora Ord.', ' Warrant', ' Cond.', ' Cod.Ch.', ' Moneda Liquidacion',
       ' Tipo de Cambio', ' Sin Recurso', ' No a la Orden', 'Vto.',
       'Nombre Banco', 'N. Sucursal', 'Pyme', 'Primera.Neg',
       'Tipo Instrumento', 'Custodio/Registro', 'Echeqid', 'Plaza Cheque',
       'Nro.Cta.Libr.', 'IVA Der.Mer.', 'CUIT Librador/Deudor', 'CUIT PyME',
       'Liquidador Compra', 'Caracter', 'CUIT Benef.', 'Razón Benef.',
       'Cond. Pyme', 'Razón Librador/Deudor\r', 'dias_entre_fechas', 'SGR',
       'Segmento', 'Periodo'],
      dtype='object')

In [50]:
from sqlalchemy import create_engine

# Especificamos los detalles de la conexión a la base de datos PostgreSQL
usuario = 'postgres'
contraseña = 'Camila1995.'
host = 'localhost'
puerto = '5432'
base_de_datos = 'mav2024'

# Creamos la URL de conexión a la base de datos PostgreSQL
url_de_conexion = f'postgresql://{usuario}:{contraseña}@{host}:{puerto}/{base_de_datos}'

# Creamos una instancia del motor (engine) de SQLAlchemy
engine = create_engine(url_de_conexion)

# Guardamos el nuevo DataFrame en la tabla existente en la base de datos
nombre_tabla = 'mav2024'  # Especifica el nombre de la tabla donde deseas agregar los datos
df.to_sql(nombre_tabla, engine, if_exists='append', index=False)

4